In [1]:
import requests
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import os
from dotenv import load_dotenv

In [2]:
print(os.path.exists('/.dockerenv'))

False


In [3]:
load_dotenv()
api_key = os.getenv('ALPHA_VANTAGE_API_KEY') # Alpha Vantage API Key
if not api_key:
    raise ValueError("API key is missing. Set the Alpha_Vantage_API_KEY environment variable in the .env file.")

ts = TimeSeries(key = api_key, output_format = 'pandas')
tickers = ["AAPL", "GOOG", "MSFT", "AMZN", "META", "NVDA", "TSLA", "NFLX", "AMD", "PLTR"]

In [4]:
all_data = []

for ticker in tickers:
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize = 'compact')
        data.reset_index(inplace = True)
        data['stock'] = ticker
        all_data.append(data)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

In [5]:
combined_df = pd.concat(all_data, ignore_index = True)

combined_df.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'stock']
combined_df = combined_df[['stock', 'date', 'open', 'high', 'low', 'close', 'volume']]

combined_df.isna().sum()

stock     0
date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [6]:
names = ["Apple", "Google", "Microsoft", "Amazon", "Meta", "Nvidia", "Tesla", "Netflix", "AMD", "Palantir"]
ticker_to_name = dict(zip(tickers, names))

In [7]:
combined_df['stock'] = combined_df['stock'].replace(ticker_to_name)

In [8]:
combined_df

,stock,date,open,high,low,close,volume
0,Apple,2025-05-15,210.95,212.9600,209.5400,211.45,45029473.0
1,Apple,2025-05-14,212.43,213.9400,210.5801,212.33,49325825.0
2,Apple,2025-05-13,210.43,213.4000,209.0000,212.93,51909332.0
3,Apple,2025-05-12,210.97,211.2679,206.7500,210.79,63775814.0
4,Apple,2025-05-09,199.00,200.5399,197.5350,198.53,36453923.0
...,...,...,...,...,...,...,...
995,Palantir,2024-12-26,83.16,83.4900,81.3000,82.14,60547746.0
996,Palantir,2024-12-24,81.09,84.7968,81.0000,82.38,64983263.0
997,Palantir,2024-12-23,80.65,82.0200,77.4100,80.69,93819484.0
998,Palantir,2024-12-20,72.64,80.6600,71.5000,80.55,281230836.0


In [9]:
print(combined_df['open'].dtype)

float64


In [10]:
combined_df.to_csv("assets/stock_prices.csv", index = False)